# Análise de novos fatores de boosting

Nesta análise vamos partir dos conhecimentos adquiridos na análise de podas do AQE, onde descobrimos que utilizar a ordem do fator de boosting com até **cinco termos** é o método de poda mais razoável para o caso geral, o qual obteve **NDCG@24 de 76,58%**. Nesse método, foi fixado o fator de boosting de 0.100 para todos os termos, independente da base utilizada. Agora, vamos experimentar com diferentes fatores de boosting para diferentes bases de dados.

Na análise das bases de dados, identificamos também as bases de dados mais relevantes para o problema. Agora vamos utilizar as três bases de dados mais relevantes, inicialmente com os pesos estáticos e similares aos identificados durante a análise anterior, os quais estão descritos abaixo: 

1. 06_Termos_TabelaPocosANP2019: 0.1
2. 13_Lista_MWE: 0.08
3. 02_Tesauro_comTraducoesRegis: 0.1

## Carregando libs

In [ ]:
from itertools import product
from pathlib import Path
import json
import yaml
import numpy as np
import pandas as pd
import plotly.express as px

from utils.utils import get_expanded_queries, make_elasticsearch_new_aqe_queries,\
    create_new_expanded_queries, create_new_aqe_validation_dataset, create_new_aqe_metrics,\
    expanded_with_aqe_boost_order, make_elasticsearch_new_queries, adjust_new_expanded_queries,\
    create_new_validation_dataset, create_new_metrics

## Carregando as configurações e bases dados

In [ ]:
with open("../conf/config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

In [ ]:
with open("../../dados/regis/regis_queries.json", 'r') as regis_file:
    regis_queries = json.load(regis_file)

In [ ]:
regis_queries = get_expanded_queries(regis_queries)
regis_queries[:2]

In [ ]:
ground_truth = pd.read_csv(
    "../../dados/regis/regis_ground_truth.csv"
).rename(
    columns={"relevance": "relevance_ground_truth"}
)
ground_truth.head()

In [ ]:
data_path = Path("../../dados/vocabulario_oil_gas")
databases = dict()

with open(data_path.joinpath("01_DicionarioPetroleo_Curado_ComSinonimos.csv"), "r", encoding='unicode_escape') as f:
    databases["01_DicionarioPetroleo_Curado_ComSinonimos"] = f.read().replace(";", "\n").split("\n")
    databases["01_DicionarioPetroleo_Curado_ComSinonimos"] = [e.strip().lower() for e in databases["01_DicionarioPetroleo_Curado_ComSinonimos"]]

with open(data_path.joinpath("02_Tesauro_comTraducoesRegis.csv"), "r", encoding='unicode_escape') as f:
    databases["02_Tesauro_comTraducoesRegis"] = f.read().replace("#", ";").replace("\t", "").replace(";", "\n").split("\n")
    databases["02_Tesauro_comTraducoesRegis"] = [e.strip().lower() for e in databases["02_Tesauro_comTraducoesRegis"] if e != ""]

with open(data_path.joinpath("03_ListaCurada.csv"), "r", encoding='unicode_escape') as f:
    databases["03_ListaCurada"] = f.read().split("\n")
    databases["03_ListaCurada"] = [e.strip().lower() for e in databases["03_ListaCurada"] if e != ""]

with open(data_path.joinpath("05_InstanciasBDIEP_Ativo_Bloco_Campo.csv"), "r", encoding='unicode_escape') as f:
    databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] = f.read().replace(";", "\n").split("\n")
    databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] = [e.strip().lower() for e in databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] if e != ""]

with open(data_path.joinpath("06_Termos_TabelaPocosANP2019.csv"), "r", encoding='unicode_escape') as f:
    databases["06_Termos_TabelaPocosANP2019"] = f.read().replace(";", "\n").split("\n")
    databases["06_Termos_TabelaPocosANP2019"] = [e.strip().lower() for e in databases["06_Termos_TabelaPocosANP2019"] if e != ""]

with open(data_path.joinpath("07_Pocos_TabelaPocosANP2019.csv"), "r", encoding='unicode_escape') as f:
    databases["07_Pocos_TabelaPocosANP2019"] = f.read().replace(";", "\n").split("\n")
    databases["07_Pocos_TabelaPocosANP2019"] = [e.strip().lower() for e in databases["07_Pocos_TabelaPocosANP2019"] if e != ""]

with open(data_path.joinpath("08_Pocos_BDIEP_com2ou3_Siglas.csv"), "r", encoding='utf-8-sig') as f:
    databases["08_Pocos_BDIEP_com2ou3_Siglas"] = f.read().replace(";", "\n").split("\n")
    databases["08_Pocos_BDIEP_com2ou3_Siglas"] = [e.strip().lower() for e in databases["08_Pocos_BDIEP_com2ou3_Siglas"] if e != ""]

with open(data_path.joinpath("09_Glossario_ANP.csv"), "r", encoding='unicode_escape') as f:
    databases["09_Glossario_ANP"] = f.read().replace(";", "\n").split("\n")
    databases["09_Glossario_ANP"] = [e.strip().lower() for e in databases["09_Glossario_ANP"] if e != ""]

with open(data_path.joinpath("10_List_of_abbreviations_curada.csv"), "r", encoding='unicode_escape') as f:
    databases["10_List_of_abbreviations_curada"] = f.read().replace(";", "\n").split("\n")
    databases["10_List_of_abbreviations_curada"] = [e.strip().lower() for e in databases["10_List_of_abbreviations_curada"] if e != ""]

with open(data_path.joinpath("11_Lista_Feita_a_Mao.csv"), "r", encoding='unicode_escape') as f:
    databases["11_Lista_Feita_a_Mao"] = f.read().replace(",", ";").replace(";", "\n").split("\n")
    databases["11_Lista_Feita_a_Mao"] = [e.strip().lower() for e in databases["11_Lista_Feita_a_Mao"] if e != ""]

with open(data_path.joinpath("12_Partex_Acronymis_Oil_Gas.csv"), "r") as f:
    databases["12_Partex_Acronymis_Oil_Gas"] = f.read().replace(";", "\n").split("\n")
    databases["12_Partex_Acronymis_Oil_Gas"] = [e.strip().lower() for e in databases["12_Partex_Acronymis_Oil_Gas"] if e != ""]

with open(data_path.joinpath("13_Lista_MWE.txt"), "r") as f:
    databases["13_Lista_MWE"] = f.read().replace(",", "\n").split("\n")
    databases["13_Lista_MWE"] = [e.strip().lower() for e in databases["13_Lista_MWE"] if e != ""]

In [ ]:
databases_factors = {
    "02_Tesauro_comTraducoesRegis": np.arange(0.02, 0.11, 0.02).tolist(),
    "06_Termos_TabelaPocosANP2019": np.arange(0.02, 0.11, 0.02).tolist(),
    "13_Lista_MWE": np.arange(0.02, 0.11, 0.02).tolist(),
}

## Criando queries com bases mais relevantes e podas baseadas nos fatores de boosting do AQE

Aqui vamos experimentar utilizar as bases mais relevantes, descritas anteriormente em conjunto com uma poda com os cinco termos de maior fator estabelecidos pelo AQE.

In [ ]:
all_expanded_queries = list()
for query in regis_queries:
    new_expanded_queries = create_new_expanded_queries(
        query["expanded_query"],
        expansion=expanded_with_aqe_boost_order,
        num_termos=[5]
    )

    for num_termos, new_expanded_query in new_expanded_queries:
        q = query.copy()
        q["expanded_query"] = new_expanded_query
        all_expanded_queries.append(q)

databases_factors_products = [
    dict(zip(databases_factors.keys(), dbf_values)) for
    dbf_values in product(*databases_factors.values())
]
all_adjusted_expanded_queries = list()
for dbf in databases_factors_products:
    dbf_queries = adjust_new_expanded_queries(
        {k: v for k, v in databases.items() if k in databases_factors.keys()},
        all_expanded_queries,
        dbf,
        False
    )
    dbf_queries = [
        dbf_query | {"boost_factor_{}".format(dbf_k): dbf_v for dbf_k, dbf_v in dbf.items()} for
        dbf_query in dbf_queries]
    all_adjusted_expanded_queries.extend(dbf_queries)
all_adjusted_expanded_queries[:2]

### Realizando consultas no Elasticsearch

Em posse das queries que utilizam cinco termos vamos criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
ranking_result_df = make_elasticsearch_new_aqe_queries(
    all_adjusted_expanded_queries,
    cfg,
    num_docs=24,
    attrs=[
        'query_id',
        'boost_factor_02_Tesauro_comTraducoesRegis',
        'boost_factor_06_Termos_TabelaPocosANP2019',
        'boost_factor_13_Lista_MWE'
    ]
)
ranking_result_df.head()

In [ ]:
boost_cols = [
    "boost_factor_02_Tesauro_comTraducoesRegis",
    "boost_factor_06_Termos_TabelaPocosANP2019",
    "boost_factor_13_Lista_MWE"
]

validation_dataset = create_new_aqe_validation_dataset(ranking_result_df, ground_truth, boost_cols)
validation_dataset.head()

### Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e quantidade de termos derivados e visualizar os resultados.

#### Criando métricas

In [ ]:
boost_cols=[
    "boost_factor_02_Tesauro_comTraducoesRegis",
    "boost_factor_06_Termos_TabelaPocosANP2019",
    "boost_factor_13_Lista_MWE"
]
metrics_df = create_new_aqe_metrics(validation_dataset, boost_cols=boost_cols)
metrics_df.head()

#### Avaliando métricas

Vamos agora avaliar as métricas. Vamos utilizar as seguintes métricas:

* ndcg - Normalized Discounted Cumulative Gain
* map - Mean Average Precision
* eval_prop - Proporção de documentos avaliados

Vejamos qual a melhor quantidade de termos derivados para cada query:

In [ ]:
boost_cols=["boost_factor_02_Tesauro_comTraducoesRegis",
            "boost_factor_06_Termos_TabelaPocosANP2019",
            "boost_factor_13_Lista_MWE"]
    
data_viz = metrics_df.melt(
    id_vars=["query_id", "ndcg@24"], value_vars=boost_cols,
    var_name="database", value_name="boost_factor"
).groupby(
    ["database", "boost_factor"]
).agg(
    ndcg_mean=("ndcg@24", "mean")
).reset_index()


fig = px.line(
    data_viz,
    x="boost_factor",
    y="ndcg_mean",
    color="database",
    markers=True,
    labels={
        "boost_factor": "Fator de boost",
        "ndcg_mean": "NDCG@24 médio",
    }
)
fig.show()

Podemos ver que para o tesauro, o melhor fator de boost é de 0,02, enquanto para a lista MWE e a tabela de poços traz o melhor fator de boost em 0,06.

Vejamos se as combinações com melhores NDCGs@24 médios concordam com esses valores.

In [ ]:
metrics_df.groupby(
    boost_cols
).agg(
    ndcg_mean=("ndcg@24", "mean"),
).reset_index(
).rename(
    columns={"ndcg_mean": "ndcg@24 mean"}
).sort_values(
    "ndcg@24 mean", ascending=False
).head(5)

Percebemos que quando usamos os fatores em conjunto, os valores são ligeiramente diferentes, onde a tabela de poços trouxe o melhor fator de boost em 0,08, enquanto os demais permaneceram os mesmos.

O valor de NDCG@24 encontrado anteriormente, utilizando todas as bases de dados, com pesos derivados do AQE e sem podas foi de 0.779687, que é maior que o maior valor encontrado utilizando estes valores.

Vamos agora experimentar com os mesmos fatores utilizados no experimento de análise da base de dados, utilizando o fator de multiplicação ao invés de estático por base, para identificar se a poda de fato traz um ganho.

In [ ]:
databases_factors = {
    "01_DicionarioPetroleo_Curado_ComSinonimos": 0.03,
    "02_Tesauro_comTraducoesRegis": 0.1,
    "03_ListaCurada": 0.0,
    "05_InstanciasBDIEP_Ativo_Bloco_Campo": 0.01,
    "06_Termos_TabelaPocosANP2019": 0.1,
    "07_Pocos_TabelaPocosANP2019": 0.0,
    "08_Pocos_BDIEP_com2ou3_Siglas": 0.0,
    "09_Glossario_ANP": 0.33,
    "10_List_of_abbreviations_curada": 0.11,
    "11_Lista_Feita_a_Mao": 0.0,
    "12_Partex_Acronymis_Oil_Gas": 0.12,
    "13_Lista_MWE": 0.08,
}

In [ ]:
all_expanded_queries = list()
for query in regis_queries:
    new_expanded_queries = adjust_new_expanded_queries(
        databases, [query], databases_factors, True
    )[0]

    new_expanded_queries = create_new_expanded_queries(
        new_expanded_queries["expanded_query"],
        expansion=expanded_with_aqe_boost_order,
        num_termos=[5],
        factor=None
    )
   
    for i, new_expanded_query in new_expanded_queries:
        q = query.copy()
        q["expanded_query"] = new_expanded_query
        all_expanded_queries.append(q)


all_expanded_queries[:2]

In [ ]:
ranking_result_df = make_elasticsearch_new_queries(all_expanded_queries, cfg, 24)
ranking_result_df.head()


In [ ]:
validation_dataset = create_new_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

In [ ]:
metrics_df = create_new_metrics(validation_dataset)
metrics_df.head()

In [ ]:
metrics_df["ndcg@24"].mean()

Podemos ver que a métrica atingida por esse método é o que trouxe melhores resultados até então.

## Conclusão

Nesta análise duas abordagens foram testadas: experimentar diferentes pesos com as três bases mais relevantes encontradas na análise das base de dados e utilizar os fatores de multiplicação de boosting, também encontrado na análise das bases de dados, ambos seguidos de uma poda com 5 termos. 

Apesar de atingir uma métrica de ranking superior ao Elasticsearch puro, a primeira abordagem resultou em uma métrica inferior às encontradas em análises anteriores, com um **NDCG@24 de 77,72%**, logo foi descartada. Já a segunda abordagem trouxe os melhores valores encontrados até então, atingingo um **NDCG@24 de 78,33%**. A nível comparativo, o **Elasticsearch puro** resulta em um **NDCG@24 de 76,58%**, utilizar **apenas os fatores de multiplicação dos boosts** resulta em um **NDCG@24 de 77,97%**, utilizar o **mecanismo de poda com cinco termos derivados** resulta em um **NDCG@24 de 76,58%**, enquanto utilizar os **fatores de multiplicação dos boosts em conjunto com o mecanismo de poda com cinco termos derivados** (segunda abordagem) resulta em um **NDCG@24 de 78,33%**. Os fatores de multiplicação da segunda abordagem, os quais multiplicam os fatores de boost retornados pelo AQE, estão descritos a seguir:

* 01_DicionarioPetroleo_Curado_ComSinonimos: 0.03
* 02_Tesauro_comTraducoesRegis: 0.1
* 03_ListaCurada: 0.0
* 05_InstanciasBDIEP_Ativo_Bloco_Campo: 0.01
* 06_Termos_TabelaPocosANP2019: 0.1
* 07_Pocos_TabelaPocosANP2019: 0.0
* 08_Pocos_BDIEP_com2ou3_Siglas: 0.0
* 09_Glossario_ANP: 0.33
* 10_List_of_abbreviations_curada: 0.11
* 11_Lista_Feita_a_Mao: 0.0
* 12_Partex_Acronymis_Oil_Gas: 0.12
* 13_Lista_MWE: 0.08

Vale lembrar que a poda com cinco termos foi realizada posteriormente à multiplicação dos fatores de boost pelos pesos descritos acima, utilizando os cinco termos com maior boost para cada termo derivado. 